<h1 align='center'>10.1 GroupBy Mechanics

Split-Apply-Combine  


    In  thefirst  stage  of  the  process,  data  contained  in  a  pandas  object,  whether  a  Series,  
    Data‐Frame, or otherwise, is split into groups based on one or more keys that you provide.
    The splitting is performed on a particular axis of an object. 
    For example, a DataFramecan  be  grouped  on  its  rows  (axis=0)  or  its  columns  (axis=1).  
    
    Once  this  is  done,  afunction  is  applied  to  each  group,  producing  a  new  value.  
    
    Finally,  the  results  of  allthose function applications are combined into a result object. 
    The form of the result‐ing object will usually depend on what’s being done to the data. 

In [3]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-1.030719,-0.133071
1,a,two,0.872076,-1.912238
2,b,one,2.335207,0.192118
3,b,two,1.369926,0.236251
4,a,one,-0.121629,-0.540275


In [4]:
grouped = df['data1'].groupby(df['key1'])

This grouped variable is now a GroupBy object. It has not actually computed anythingyet  except  for  some  intermediate  data  about  the  group  key  df['key1'].  The  idea  isthat  this  object  has  all  of  the  information  needed  to  then  apply  some  operation  toeach of the groups.

In [5]:
grouped.mean()

key1
a   -0.093424
b    1.852567
Name: data1, dtype: float64

If instead we had passed multiple arrays as a list, we’d get something different:

In [18]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.576174
      two     0.872076
b     one     2.335207
      two     1.369926
Name: data1, dtype: float64

In this example, the group keys are all Series, though they could be any arrays of theright length

In [13]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [17]:
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states,years]).mean()

California  2005    0.872076
            2006    2.335207
Ohio        2005    0.169604
            2006   -0.121629
Name: data1, dtype: float64

In [22]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.093424,-0.861861
b,1.852567,0.214184


You  may  have  noticed  in  the  first  case  df.groupby('key1').mean()  that  there  is  nokey2 column in the result. Because df['key2'] is not numeric data, it is said to be anuisance  column,  which  is  therefore  excluded  from  the  result.  By  default,  all  of  thenumeric  columns  are  aggregated,

In [23]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

<b>Take note that any missing values in a group key will be excluded from the result

<h3>Iterating Over Groups

In [27]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.030719 -0.133071
1    a  two  0.872076 -1.912238
4    a  one -0.121629 -0.540275
b
  key1 key2     data1     data2
2    b  one  2.335207  0.192118
3    b  two  1.369926  0.236251


In the case of multiple keys, the first element in the tuple will be a tuple of key values:

In [30]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group.unstack())

('a', 'one')
key1   0           a
       4           a
key2   0         one
       4         one
data1  0    -1.03072
       4   -0.121629
data2  0   -0.133071
       4   -0.540275
dtype: object
('a', 'two')
key1   1           a
key2   1         two
data1  1    0.872076
data2  1    -1.91224
dtype: object
('b', 'one')
key1   2           b
key2   2         one
data1  2     2.33521
data2  2    0.192118
dtype: object
('b', 'two')
key1   3           b
key2   3         two
data1  3     1.36993
data2  3    0.236251
dtype: object


Of course, you can choose to do whatever you want with the pieces of data. A recipe you may find useful is computing a dict of the data pieces as a one-liner:

In [31]:
pieces = dict(list(df.groupby('key1')))

In [33]:
pieces['b']

,key1,key2,data1,data2
2,b,one,2.335207,0.192118
3,b,two,1.369926,0.236251


By  default  groupby  groups  on  axis=0,  but  you  can  group  on  any  of  the  other  axes.

In [35]:
grouped = df.groupby(df.dtypes, axis=1)

In [38]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.030719 -0.133071
1  0.872076 -1.912238
2  2.335207  0.192118
3  1.369926  0.236251
4 -0.121629 -0.540275
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


<h3>Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or arrayof  column  names  has  the  effect  of  column  subsetting  for  aggregation.  This  meansthat:

    df.groupby('key1')['data1']

    df.groupby('key1')[['data2']]

are syntactic sugar for:

    df['data1'].groupby(df['key1'])

    df[['data2']].groupby(df['key1'])

In [39]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.336673
     two  -1.912238
b    one   0.192118
     two   0.236251

The  object  returned  by  this  indexing  operation  is  a  grouped  DataFrame  if  a  list  orarray is passed or a grouped Series if only a single column name is passed as a scalar:

<h3>Grouping with Dicts and Series

In [40]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [41]:
people.iloc[2:3, [1, 2]] = np.nan

In [42]:
people

,a,b,c,d,e
Joe,0.896577,0.244578,-0.506562,0.364375,0.627787
Steve,0.844416,0.133873,-2.010795,-0.019360,-0.296703
Wes,2.636953,NaN,NaN,0.536806,-0.247194
Jim,-0.968746,0.939866,-0.554720,-1.183852,0.194210
Travis,0.243452,-1.650724,1.223886,1.252139,0.668501


 I  have  a  group  correspondence  for  the  columns  and  want  to  sumtogether the columns by group:

In [43]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [44]:
by_column = people.groupby(mapping, axis=1)

In [45]:
by_column.sum()

,blue,red
Joe,-0.142187,1.768942
Steve,-2.030155,0.681587
Wes,0.536806,2.389759
Jim,-1.738572,0.165330
Travis,2.476025,-0.738771


The same functionality holds for Series, which can be viewed as a fixed-size mapping:

In [46]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [47]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


<h3>Grouping with Functions

Suppose  you  wanted  to  group  by  the  length  of  the  names

In [48]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.564785,1.184443,-1.061283,-0.282671,0.574803
5,0.844416,0.133873,-2.010795,-0.019360,-0.296703
6,0.243452,-1.650724,1.223886,1.252139,0.668501


In [49]:
people

,a,b,c,d,e
Joe,0.896577,0.244578,-0.506562,0.364375,0.627787
Steve,0.844416,0.133873,-2.010795,-0.019360,-0.296703
Wes,2.636953,NaN,NaN,0.536806,-0.247194
Jim,-0.968746,0.939866,-0.554720,-1.183852,0.194210
Travis,0.243452,-1.650724,1.223886,1.252139,0.668501


Mixing functions with arrays, dicts, or Series is not a problem as everything gets con‐verted to arrays internally

In [50]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [51]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.896577  0.244578 -0.506562  0.364375 -0.247194
  two -0.968746  0.939866 -0.554720 -1.183852  0.194210
5 one  0.844416  0.133873 -2.010795 -0.019360 -0.296703
6 two  0.243452 -1.650724  1.223886  1.252139  0.668501

<h3> Grouping by Index Levels

In [59]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'JP', 'US', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('JP', 5),
            ('US', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [60]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                  JP        US        JP
tenor         1         3         5         1         3
0     -0.473086  0.199743  0.583593 -0.423826  1.101642
1     -2.475595 -1.422810  0.124219  0.905706  1.289469
2      1.048351  1.267444  0.646831  1.557963 -0.037843
3      0.692572 -1.342016 -0.429398 -1.163448  0.247010

In [61]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
